In [1]:
import geopandas as gpd
import pyarrow
import pandas as pd
import os

In [2]:

def write_file(filename, info):
  text_file = open(filename, "w")
  text_file.write(info)
  text_file.close()

In [3]:
target_2631 = pd.read_parquet('data/GPS.nosync/filter/Sonoma_2631_new', engine='pyarrow')

In [4]:
streets = gpd.read_file('data/GPS.nosync/Streets/Streets.shp')

In [5]:
speed_limits = pd.read_csv('data/GPS.nosync/Speed Surveys.csv')

In [6]:
speed_limits.columns

Index(['Road Name', 'Road ID', 'Lane Configuration', 'Travel Lane Width',
       'Paved Shoulder Width', 'Graded Shoulder Width', 'Surface Condition',
       'Parking', 'Driveways', 'Sidewalks', 'Terrain', 'School Zone',
       'Bike Route', 'Intersections', 'Posted Speed Limit',
       '85th Percentile Speed', 'PACE Range', 'Radar Date', 'Completed By',
       'Purpose', 'Postmile', 'Class', 'Lon', 'Lat', 'created_user',
       'created_date', 'last_edited_user', 'last_edited_date', 'x', 'y'],
      dtype='object')

In [8]:
speed_limits['Posted Speed Limit'].value_counts()

35.0    193
45.0    137
40.0    127
30.0    109
25.0     77
50.0     41
55.0     13
Name: Posted Speed Limit, dtype: int64

In [9]:
speed_limits['Road Name'].value_counts()

River Rd                      26
Arnold Dr                     25
Stewarts Pt Skaggs Springs    20
Occidental Rd                 17
Grove St                      14
                              ..
Alton Ln                       1
Standish Ave                   1
Kinley Dr                      1
Taylor Ave                     1
Roberts Rd                     1
Name: Road Name, Length: 261, dtype: int64

In [10]:
speed_limits[speed_limits['Road Name'] == 'Llano Rd']

,Road Name,Road ID,Lane Configuration,Travel Lane Width,Paved Shoulder Width,Graded Shoulder Width,Surface Condition,Parking,Driveways,Sidewalks,...,Postmile,Class,Lon,Lat,created_user,created_date,last_edited_user,last_edited_date,x,y
123,Llano Rd,6805,NBSB,12,3-9,0-6,Good,"E,W","1-E, 5-W",NaN,...,10.37,MaC,-122.769139,38.363225,CGALANTI,"六月 27, 2017",TPW,"二月 14, 2019",6.341131e+06,1.895034e+06
332,Llano Rd,6805,"NB,SB",10,0-1,0-2,Fair,"E,W","11-E, 6-W",None,...,11.55,MaC,-122.768794,38.380256,STAGE,"七月 6, 2016",TPW,"二月 14, 2019",6.341282e+06,1.901235e+06
333,Llano Rd,6805,"NB,SB",12,1-4,0-4,Fair,"E,W","28-E, 11-W",None,...,13.84,MaC,-122.791566,38.406722,STAGE,"七月 6, 2016",TPW,"二月 14, 2019",6.334838e+06,1.910929e+06
656,Llano Rd,6805,"NB,SB",12,1-4,0-4,Fair,"E,W","15-E, 7-W",None,...,13.10,MaC,-122.783826,38.398076,STAGE,"七月 6, 2016",TPW,"二月 14, 2019",6.337029e+06,1.907761e+06
657,Llano Rd,6805,"NB,SB",10,0-1,0-2,Fair,"E,W","11-E, 4-W",None,...,11.33,MaC,-122.768775,38.377025,STAGE,"七月 6, 2016",TPW,"二月 14, 2019",6.341277e+06,1.900058e+06


In [11]:
speed_limits[speed_limits['Road Name'] == 'Llano Rd']['Posted Speed Limit']

123     NaN
332    45.0
333    50.0
656    50.0
657    45.0
Name: Posted Speed Limit, dtype: float64

In [12]:
def show_points(filename, dataType):
  if os.path.isfile(filename):
    #open text file in read mode
    text_file = open(filename, "r")

    #read whole file to a string
    points = text_file.read()

    #close file
    text_file.close()
    data = {'col1': ['name1'], 'wkt': points}
    df = pd.DataFrame(data)
    gs = gpd.GeoSeries.from_wkt(df['wkt'])
    gdf = gpd.GeoDataFrame(df, geometry=gs, crs="EPSG:4326")
    #print(type(gs))
    if dataType == 2:
      tmp = gdf.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
      x_ = []
      y_ = []
      for data in tmp[0]:
        x_.append(data[0])
        y_.append(data[1])
      gdf = gpd.GeoDataFrame(
        pd.DataFrame(), geometry=gpd.points_from_xy(x=x_, y=y_))
    return gdf
  else:
    print("NO SUCH FILE")

def show_points_from_df(df, index, dataType):

    #read whole file to a string
    points = df.loc[index]["res"]
    #print(points)
    data = {'col1': ['name1'], 'wkt': points}
    df1 = pd.DataFrame(data)
    gs = gpd.GeoSeries.from_wkt(df1['wkt'])
    gdf = gpd.GeoDataFrame(df1, geometry=gs, crs="EPSG:4326")
    #print(type(gs))
    if dataType == 2:
      tmp = gdf.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
      x_ = []
      y_ = []
      for data in tmp[0]:
        x_.append(data[0])
        y_.append(data[1])
      gdf = gpd.GeoDataFrame(
        pd.DataFrame(), geometry=gpd.points_from_xy(x=x_, y=y_))
    return gdf


In [27]:
gdf_wkt = []
gdf_ekt = []
gdf_points = []
gdf_points_string = []
#index_list = [1332,1396,1472]
index_list = [13, 29, 187, 578, 1375, 3204, 4902, 5877,10221, 13327]
df = pd.read_csv('data/GPS.nosync/Sonoma-2631-new-points/out2.csv')
#index_list = [4, 23, 187, 370, 497, 705, 831, 1000, 1061, 1536, 1592]
for i in index_list:
  print(i)
  line_base = "data/GPS.nosync/output/sub-result-"
  points_base = "data/GPS.nosync/points/"
  file_path1 = line_base + str(i) + '/' + str(i) + "-m_wkt.txt"
  gdf_wkt.append(show_points(file_path1,1))
  # file_path2 = line_base + str(i) + '/' + str(i) + "-e_wkt.txt"
  # gdf_ekt.append(show_points(file_path2,1))
  # file_path3 = points_base + str(i) + "-points.txt"
  # gdf_points.append(show_points(file_path3 ,2))
  gdf_points_string.append(show_points_from_df(df,i, 1))
  gdf_points.append(show_points_from_df(df, i ,2))


13
29
187
578
1375
3204
4902
5877
10221
13327


In [22]:
gdf_points_string[0]

,col1,wkt,geometry
0,name1,"LINESTRING(-122.811018 38.549082,-122.81128 38...","LINESTRING (-122.81102 38.54908, -122.81128 38..."


In [28]:
import folium
from folium import plugins
for index in range(0,10):
    map2 = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12,max_zoom = 16, min_zoom=8)

    for coord in gdf_points[index]['geometry']:
        folium.Marker( location=[ coord.y, coord.x ]).add_to( map2 )
    folium.Choropleth(
        gdf_points_string[index],
        line_weight=2,
        line_color='red'
    ).add_to(map2)
    #plugins.HeatMap(heat_data).add_to(map)
    folium.Choropleth(
        gdf_wkt[index],
        line_weight=3,
        line_color='blue'
    ).add_to(map2)

    map2.save(str(index) + "tmp1.html")
#map2[0]

In [ ]:
map3 = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12,max_zoom = 14, min_zoom=10)

plugins.HeatMap(heat_data3,min_opacity=0.4,
                   radius=21, blur=12, 
                   max_zoom=4, min_zoom=8).add_to(map3)
#plugins.HeatMap(heat_data).add_to(map)
map3


In [ ]:
map4 = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12,max_zoom = 14, min_zoom=10)

plugins.HeatMap(heat_data4,min_opacity=0.4,
                   radius=21, blur=12, 
                   max_zoom=4, min_zoom=8).add_to(map4)
#plugins.HeatMap(heat_data).add_to(map)
map4

In [ ]:
map5 = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12,max_zoom = 14, min_zoom=10)

plugins.HeatMap(heat_data3,min_opacity=0.4,
                   radius=21, blur=12, 
                   max_zoom=4, min_zoom=8).add_to(map5)
#plugins.HeatMap(heat_data).add_to(map)
map5

In [ ]:
map6 = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12,max_zoom = 14, min_zoom=10)

plugins.HeatMap(heat_data6,min_opacity=0.4,
                   radius=21, blur=12, 
                   max_zoom=4, min_zoom=8).add_to(map6)
#plugins.HeatMap(heat_data).add_to(map)
map6

In [ ]:
map7 = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12,max_zoom = 14, min_zoom=10)

plugins.HeatMap(heat_data7,min_opacity=0.4,
                   radius=21, blur=12, 
                   max_zoom=4, min_zoom=8).add_to(map7)
#plugins.HeatMap(heat_data).add_to(map)
map7

In [ ]:
map2.save("26-06-26-12.html")
map3.save("26-12-26-22.html")
map4.save("26-22-27-06.html")
map5.save("27-06-27-12.html")
map6.save("27-12-27-22.html")

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(64, 64))
streets.plot(ax=ax, color='black', LineWidth = 2)
gdf2.plot(ax=ax, color='green',LineWidth = 2)
#gdf.plot(ax=ax, color='blue', LineWidth = 1)
gdf1.plot(ax=ax, color='red',LineWidth = 1)

In [ ]:
# parts = streets[1:10000]
# parts['points'] = parts.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
# len(parts)
gdf2['points'] = gdf1.apply(lambda x: [y for y in x['geometry'].coords], axis=1)
gdf1.head
gdf1

In [ ]:
import folium
map = folium.Map(location = [4,10], tiles = "Stamen Terrain", zoom_start = 3)

In [ ]:
from folium import plugins

map = folium.Map(location=[38.447135, -122.76633], tiles='OpenStreetMap',zoom_start=12)

#heat_data = [gdf2.points[0],gdf2.points[1]]
#heat_data[1] = 
heat_data = [[point[1], point[0]] for point in gdf2['points'][0] ]
# for point in gdf2['points'][1]:
#   heat_data.append([point[1],point[0]])
for points in gdf2['points']:
  for p in points:
    heat_data.append([p[1],p[0]])
heat_data
plugins.HeatMap(heat_data,min_opacity=0.5,
                   radius=4, blur=4, 
                   max_zoom=1).add_to(map)
map


In [ ]:
heat_data

In [ ]:
map.save("index-1-14-13-13.html")

In [ ]:
plugins.HeatMap(heat_data).add_to(map)
map